## Add new behavior sessions to metadata sheet from data folders

* Notebook to automatically add basepath, basename and video name to metadata.csv for SNLab behavior sessions


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

### Set paths to dataset and Metadata 


In [21]:
dataset_path = r"Y:\laura_berkowitz\behavior_validation\appps1_cpp\data\cohort6"
metadata_path = r"Y:\laura_berkowitz\behavior_validation\appps1_cpp\metadata.csv"

## Find all folders in Cohort 6 and interate through. If a video exists, then add to metadata dataframe. 

In [22]:
metadata_df = pd.read_csv(metadata_path)
datafolders = glob.glob(dataset_path+'**\*\*', recursive=True)

## Iterater through each folder, if a video exists, add each video to the metadata dataframe, including basepath, subid, basename and video name. 

In [23]:

for basepath in datafolders: 
    # check if the folder contains a video file 
    vid_files = glob.glob(basepath + '\*.avi')
    basename = os.path.basename(basepath)
    subid = basename.split('_')[0]

    # if the folder contains a video file & the subid is not already in the metadata dataframe
    if (len(vid_files) > 0) & (basepath not in metadata_df['basepath'].values):
        # add each video file to the metadata dataframe
        for vid in vid_files:
            metadata_df = metadata_df.append({'subid': subid, 'vidname': os.path.basename(vid), 'basepath': basepath,'basename':basename}, ignore_index=True)
        

C:\Users\schafferlab\AppData\Local\Temp\ipykernel_17912\915143459.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append({'subid': subid, 'vidname': os.path.basename(vid), 'basepath': basepath,'basename':basename}, ignore_index=True)
C:\Users\schafferlab\AppData\Local\Temp\ipykernel_17912\915143459.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append({'subid': subid, 'vidname': os.path.basename(vid), 'basepath': basepath,'basename':basename}, ignore_index=True)
C:\Users\schafferlab\AppData\Local\Temp\ipykernel_17912\915143459.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata_df = metadata_df.append({'subid': subid, 'vidname': os.path.basename(vid), 

## Save the updated metadata dataframe back to folder

In [24]:
## Save the metadata to a csv file
metadata_df.to_csv(metadata_path, index=False)